In [ ]:
import random
import numpy as np
import keras.utils
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, LSTM, Dense

We generate a sequence in the following data format:
* *x* and *y* describing a position in a grid of `100 x 100`
* *c* describing a control status with 3 possible states (0 = starting, 1 = holding, 2 = pausing)

State transitions follow this diagram:

```
+-+     +-+     +-+
|0| --> |1| --> |2|
+++  ^  +++  ^  +++
 ^   |   |   |   |
 |   |   |   |   |
 +---+---+   +---+
 |               |
 |               |
 +---------------+
```

This results in `100 x 100 x 3 = 30.000` possible one-hot encoded values ranging from 1 - 30.000.

In [ ]:
STATES_COUNT = 3

STATE_STARTING = 0
STATE_HOLDING = 1
STATE_PAUSING = 2

DEFAULT_POSITION = [0, 0]


def random_position(grid_size):
    return [random.randint(0, grid_size - 1) for _ in range(2)]
    

def next_state(previous_state):
    if previous_state == STATE_STARTING:
        next_state = STATE_HOLDING
    elif previous_state == STATE_HOLDING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_HOLDING,
            STATE_PAUSING
        ])
    elif previous_state == STATE_PAUSING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    else:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    return next_state
    

def generate_sequence(grid_size, seq_len):
    sequence = []
    current_state = None
    current_position = DEFAULT_POSITION
    for i in range(seq_len):
        current_state = next_state(current_state)
        if current_state == STATE_STARTING:
            current_position = random_position(grid_size)
        elif current_state == STATE_PAUSING:
            current_position = DEFAULT_POSITION
        feature_vector = np.concatenate([current_position, [current_state]])
        sequence.append(feature_vector)
    return sequence


def generate_alternative_sequence(seq, grid_size):
    sequence = generate_sequence(grid_size, len(seq))
    current_position = None
    for i in range(len(seq)):
        if sequence[i][2] == STATE_STARTING:
            # "react" to other sequence by flipping it
            if seq[i][2] != STATE_PAUSING:
                current_position = [seq[i][1], seq[i][0]]
                sequence[i][0] = current_position[0]
                sequence[i][1] = current_position[1]
            else:
                current_position= None
        elif sequence[i][2] == STATE_HOLDING:
            if current_position:
                sequence[i][0] = current_position[0]
                sequence[i][1] = current_position[1]
    return sequence
    

def encode_sequence(seq, grid_size, pad=False):
    # Encode 3-d vector in index values
    m = np.zeros((grid_size, grid_size, STATES_COUNT))
    indexed = [np.ravel_multi_index(vector, m.shape) + 1 for vector in seq]
    if pad:
        indexed = [0] + indexed[:-1]
    n_tokens = (grid_size * grid_size * STATES_COUNT) + 1
    hot_encoded = keras.utils.to_categorical(indexed, num_classes=n_tokens)
    return hot_encoded


def decode_sequence(seq, grid_size):
    m = np.zeros((grid_size, grid_size, STATES_COUNT))
    one_hot_decoded = [np.argmax(vector) for vector in seq]
    decoded = [np.unravel_index((val - 1 if val > 0 else 0), m.shape) for val in one_hot_decoded]
    return decoded


def generate_dataset(grid_size, n_in, n_out, n_samples):
    src_data, start_data, target_data = [], [], []
    for i in range(n_samples):
        # Generate source sequence
        src = generate_sequence(grid_size, n_in)
        src_encoded = encode_sequence(src, grid_size)
        # Generate target sequence
        target = generate_alternative_sequence(src[:n_out], grid_size)
        target_encoded = encode_sequence(target, grid_size)
        # Generated target input sequence, begin with start symbol 0
        start_encoded = encode_sequence(target, grid_size, pad=True)
        # ... add to dataset
        src_data.append(src_encoded)
        start_data.append(start_encoded)
        target_data.append(target_encoded)
        if i % 10000 == 0 and i > 0:
            print("Generated sample no. #%d" % i)
    return np.array(src_data), np.array(start_data), np.array(target_data)

In [ ]:
STATES_COUNT = 3

def generate_sequence(length, n_unique):
    return [random.randint(1, n_unique-1) for _ in range(length)]


def generate_dataset(cardinality, n_in, n_out, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define target sequence
        target = source[:n_out]
        target.reverse()
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = keras.utils.to_categorical([source], num_classes=cardinality)
        tar_encoded = keras.utils.to_categorical([target], num_classes=cardinality)
        tar2_encoded = keras.utils.to_categorical([target_in], num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)

In [ ]:
def define_models(grid_size, latent_dim, n_tokens):
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, n_tokens))
    encoder = LSTM(latent_dim,
                   dropout=0.2,
                   return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, n_tokens))
    decoder_lstm = LSTM(latent_dim,
                        return_sequences=True,
                        return_state=True)
    decoder_outputs, _, _ = decoder_lstm(
        decoder_inputs,
        initial_state=encoder_states)
    decoder_dense = Dense(n_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Define sampling models
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # Define inference decoder
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    # Return all models
    return model, encoder_model, decoder_model


def train(model, encoder_input, decoder_input, decoder_target, epochs, batch_size):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    model.summary()
    
    history = model.fit([encoder_input, decoder_input], decoder_target,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)
    return history


def sample(encoder_model, decoder_model, input_seq, num_decoder_tokens, n_steps):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate an empty target sequence
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Sampling loop for a batch of sequences
    sequence = []
    for t in range(n_steps):
        # Predict next vector
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        # Store prediction to new sequence
        sequence.append(output_tokens[0, 0, :])
        # Update states
        states_value = [h, c]
        # Update target sequence
        target_seq = output_tokens
    return np.array(sequence)


def evaluate(encoder_model, decoder_model, evaluation_total, n_tokens, n_in, n_out, grid_size):
    evaluation_correct = 0
    for _ in range(evaluation_total):
        # Generate a test dataset
        encoder_test, decoder_test, target_test = generate_dataset(
            grid_size, n_in, n_out, 1)
        # Sample some sequences with out trained model
        target = sample(encoder_model, decoder_model, encoder_test, n_tokens, n_out)
        if np.array_equal(
            decode_sequence(target_test[0], grid_size),
            decode_sequence(target, grid_size)):
            evaluation_correct += 1
    print("Accuracy: %.2f%%" % (
        float(evaluation_correct) / float(evaluation_total) * 100.0))

In [ ]:
grid_size = 3 # How large is the grid of our x/y vectors

n_in = 5 # Length of an input sequence
n_out = 5 # Length of an output sequence

n_dataset_samples = 100000 # Number of samples to train on
latent_dim = 256 # Latent dimensionality of the encoding space
batch_size = 512 # Batch size for training
epochs = 10 # Number of epochs to train for

n_evaluation = 100 # Number of samples to evaluate
n_samples = 10 # Number of samples to generate as an example

# All x/y positions in grid * state variants + 1 start symbol
n_tokens = (grid_size * grid_size * STATES_COUNT) + 1

# Print current configuration
print("=======================================")
print("Number of dataset samples:\t %d" % n_dataset_samples)
print("Number of unique tokens:\t %d" % n_tokens)
print("_______________________________________")
print("Sequence length for inputs:\t %d" % n_in)
print("Sequence length for outputs:\t %d" % n_out)
print("_______________________________________")
print("Epochs:\t\t\t\t %d" % epochs)
print("Batch size:\t\t\t %d" % batch_size)
print("Latent space dimension:\t\t %d" % latent_dim)
print("=======================================")

In [ ]:
# Generate a simulated dataset
encoder_input_data, decoder_input_data, decoder_target_data = generate_dataset(
    grid_size, n_in, n_out, n_dataset_samples)

print("Done! Shapes:", encoder_input_data.shape,
      decoder_input_data.shape,
      decoder_target_data.shape)

In [ ]:
generate_dataset(grid_size, n_in, n_out, 1)

In [ ]:
# Define the models
model, encoder_model, decoder_model = define_models(
    grid_size,
    latent_dim,
    n_tokens)

# Train the model
history = train(model,
                encoder_input_data,
                decoder_input_data,
                decoder_target_data,
                epochs,
                batch_size)

In [ ]:
# Evaluation
evaluate(encoder_model,
         decoder_model,
         n_evaluation,
         n_tokens,
         n_in,
         n_out,
         grid_size)

In [ ]:
# Sample a few examples
for i in range(n_samples):
    # Generate a sample dataset
    encoder_test, decoder_test, target_test = generate_dataset(
        grid_size,
        n_in,
        n_out,
        1)
    
    # Sample some sequences with out trained model
    target = sample(encoder_model,
                    decoder_model,
                    encoder_test,
                    n_tokens,
                    n_out)
    
    # Print it!
    print('Sample #%i:\nencoder_test=%s\ntarget_test=%s\ntarget=%s\n' % (
        i,
        decode_sequence(encoder_test[0], grid_size),
        decode_sequence(target_test[0], grid_size),
        decode_sequence(target, grid_size)))

In [ ]:
# Print a plot
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

plt.show()